#### @prompt example

In [1]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])
FMP_KEY = os.environ['FMP_KEY']


## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




### Attempting to write a Small Agent

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [17]:
@tool
def get_performance(group:str) -> list:
    """ Return performance by sector or industyr industry performance for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group=group, metric='performance', provider='finviz').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  5 stock performers for last week and last month for a given sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=5)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  5 stock performers for last week and last month for an industry"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Industry' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation metrics for a company"""
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
#get_valuation_for_sectors.invoke('Utilities')
#get_valuation_for_company.invoke('AAPL')
#get_valuation_for_industries.invoke('Medical Distribution')
get_best_stock_performers_for_industry('Medical Distribution')

/tmp/ipykernel_19520/515576490.py:60: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_best_stock_performers_for_industry('Medical Distribution')


'[{"symbol":"COR","name":"Cencora Inc","country":"USA","sector":"Healthcare","industry":"Medical Distribution","market_cap":53980000000.0,"price":278.39,"change_percent":-0.0379,"volume":4331792,"price_to_earnings":39.61},{"symbol":"MCK","name":"Mckesson Corporation","country":"USA","sector":"Healthcare","industry":"Medical Distribution","market_cap":85610000000.0,"price":683.11,"change_percent":-0.0472,"volume":3081849,"price_to_earnings":31.31}]'

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock financial analys, but don't know current events. Use the toools as much as you can to retrieve informations",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
tools = [get_performance, get_strong_buy_for_sector, get_best_stock_performers_for_sector, get_valuation_for_sectors, get_valuation_for_industries,get_valuation_for_company]
llm_with_tools = llm.bind_tools(tools)

In [6]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

### Chat Memory

In [7]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
#from langchain_core.messages import count_tokens_approximately
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [8]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,  # Required for token counting
    max_token_limit=14000,  # Leave buffer for functions + responses
    memory_key="chat_history",  # Must match your prompt's key
    return_messages=True
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/tmp/ipykernel_19520/1189149751.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [ ]:
input1 = "Find the sector that has shown a constant postive performance across quarter, month and week.Once you get it, please extract its valuation metrics and the top performers for this sector.Once you find the top performenrs, get their valuation metrics, compare them with the sector and summarize which companies are doing better than the sector"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

### Let's try to do another approach, We start with the industry, find the ones that are performing better than the industry and narrow down by criteria

In [12]:
input1 = '''First, analyze the available data to find an industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes. Second, once you have identified the industry, extract its relevant valuation metrics (e.g., P/E, P/B, EV/EBITDA). Third, analyze the individual companies within the identified industry to determine the top performers based on relevant metrics like stock price appreciation and revenue growth. Fourth, extract the valuation metrics of the top performers using the same metrics as the industry. Finally, compare the valuation metrics of the top performers to the valuation metrics of the industry to determine if they are overvalued, undervalued, or fairly valued relative to the broader industry.'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_performance` with `{'group': 'industry'}`


[{"name":"Oil & Gas Drilling","performance_1d":-0.1525,"performance_1w":-0.2647,"performance_1m":-0.2081,"performance_3m":-0.4135,"performance_6m":-0.445,"performance_1y":-0.5735,"performance_ytd":-0.3924,"analyst_recommendation":1.91,"volume":125480000,"volume_average":57070000,"volume_relative":2.2},{"name":"Copper","performance_1d":-0.1102,"performance_1w":-0.1985,"performance_1m":-0.1744,"performance_3m":-0.1844,"performance_6m":-0.3699,"performance_1y":-0.3356,"performance_ytd":-0.1796,"analyst_recommendation":2.73,"volume":84920000,"volume_average":36090000,"volume_relative":2.35},{"name":"Aluminum","performance_1d":-0.0947,"performance_1w":-0.1894,"performance_1m":-0.2378,"performance_3m":-0.3024,"performance_6m":-0.35,"performance_1y":-0.3831,"performance_ytd":-0.3009,"analyst_recommendation":1.77,"volume":20180000,"volume_average":8640000,"volume_relative":2.34},{"name":"Compute

OpenBBError: 
[Unexpected Error] -> ValueError -> Invalid filter option 'Medical Distribution'. Possible filter options: ['Any', 'Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']

In [ ]:
input1 = "Using the recently found strong buys for Utilities , find their valuation metrics and compare it with the Utilities sector valuation metrics"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})